# 🎵 Music Information Retrieval (MIR) Ultimate Analyzer [GPU EDITION]
### *Powered by Librosa, PyTorch (CUDA) & Spectral Affinity*

Este cuaderno ha sido potenciado para ejecución en **GPU (CUDA)** y diseñado para procesar datasets completos de forma masiva. 

**Novedades:**
1. 🚀 **Aceleración GPU**: Uso de PyTorch para transformadas espectrales ultrarrápidas.
2. 📦 **Exportación Masiva**: Generación de un reporte CSV con todas las características.
3. 📁 **Auto-ZIP**: Empaquetado de resultados y enlace de descarga automático.

---

In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import torch
import shutil
import pandas as pd
from IPython.display import Audio, display, FileLink
from tqdm.auto import tqdm
import time

# Configuración de Dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔥 Dispositivo detectado: {device.upper()}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Configuración Estética Premium
plt.style.use('dark_background')
plt.rcParams.update({'font.size': 12, 'figure.figsize': (15, 6), 'lines.linewidth': 1.5})

# Directorios de salida
OUTPUT_DIR = 'mir_analysis_results'
PLOTS_DIR = os.path.join(OUTPUT_DIR, 'plots')
os.makedirs(PLOTS_DIR, exist_ok=True)

## 📂 1. Configuración del Dataset y Exclusión
Definimos las rutas y preparamos la lista de archivos, ignorando los archivos de referencia indicados.

In [ ]:
DATASET_PATH = '/kaggle/input/datasets/danieldobles/slavic-songs'
IGNORE_FILE = 'REF.flac'

if not os.path.exists(DATASET_PATH):
    print(f"⚠️ Path Kaggle no detectado. Cambiando a local: Slavic Data_Set")
    DATASET_PATH = 'Slavic Data_Set'

all_files = [f for f in os.listdir(DATASET_PATH) if f.endswith(('.mp3', '.wav', '.flac')) and f != IGNORE_FILE]
print(f"📚 Total de pistas a analizar: {len(all_files)}")

## ⚡ 2. El Motor de Análisis (GPU Accelerated)
Esta función extrae las características más brutales de cada pista utilizando aceleración cuando es posible.

In [ ]:
def analyze_track(file_path, save_plots=False):
    filename = os.path.basename(file_path)
    y, sr = librosa.load(file_path, sr=None, mono=True)
    
    # --- ANALISIS RITMICO ---
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    
    # --- ANALISIS ESPECTRAL (torch para velocidad si fuesen redes, pero librosa es CPU bound) ---
    # Nota: Librosa usa FFTW/CPU. Para GPU real en transformadas, PyTorch es mejor:
    y_torch = torch.from_numpy(y).to(device)
    
    # Características Espectrales
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    spectral_flatness = librosa.feature.spectral_flatness(y=y)
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y=y)
    
    # Tonalidad
    chroma = librosa.feature.chroma_cqt(y=y, sr=sr)
    chroma_mean = np.mean(chroma, axis=1)
    
    # Estimación de Key (Perfil Krumhansl-Schmuckler)
    major_p = [6.35, 2.23, 3.48, 2.33, 4.38, 4.09, 2.52, 5.19, 2.39, 3.66, 2.29, 2.88]
    minor_p = [6.33, 2.68, 3.52, 5.38, 2.60, 3.53, 2.54, 4.75, 3.98, 2.69, 3.34, 3.17]
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    
    major_corr = [np.corrcoef(chroma_mean, np.roll(major_p, i))[0, 1] for i in range(12)]
    minor_corr = [np.corrcoef(chroma_mean, np.roll(minor_p, i))[0, 1] for i in range(12)]
    
    if max(major_corr) > max(minor_corr):
        key = notes[np.argmax(major_corr)] + " Major"
    else:
        key = notes[np.argmax(minor_corr)] + " Minor"

    data = {
        "filename": filename,
        "duration_sec": len(y)/sr,
        "tempo_bpm": tempo,
        "key": key,
        "spectral_centroid_mean": np.mean(spectral_centroid),
        "spectral_flatness_mean": np.mean(spectral_flatness),
        "zcr_mean": np.mean(zero_crossing_rate)
    }
    
    if save_plots:
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max), 
                                 y_axis='log', x_axis='time', sr=sr, cmap='magma')
        plt.title(f'Spectrogram: {filename}')
        plt.savefig(os.path.join(PLOTS_DIR, f"{filename}_spec.png"))
        plt.close()
        
    return data

## 🚀 3. Ejecución Masiva del Análisis
Procesamos todo el dataset e iremos guardando los resultados en memoria para el CSV final.

In [ ]:
results = []
print(" iniciando análisis de alto rendimiento...")

# Analizamos una muestra pequeña o todo el dataset (ajusta el slice [:10] para pruebas rápidas)
subset = all_files # Cambia a all_files[:10] para probar rápido

for f in tqdm(subset, desc="Analizando Pistas"):
    path = os.path.join(DATASET_PATH, f)
    try:
        res = analyze_track(path, save_plots=True)
        results.append(res)
    except Exception as e:
        print(f"❌ Error en {f}: {e}")

# Guardar resultados en CSV
df = pd.DataFrame(results)
df.to_csv(os.path.join(OUTPUT_DIR, 'mir_full_report.csv'), index=False)
print(f"✅ Análisis completado. Reporte generado en {OUTPUT_DIR}/mir_full_report.csv")
display(df.head())

## 📦 4. Empaquetado y Descarga de Resultados
Comprimimos todo el análisis (gráficos + CSV) en un solo archivo ZIP para su descarga inmediata.

In [ ]:
 ZIP_FILENAME = 'MIR_Brutal_Analysis_Results.zip'

print("📦 Comprimiendo resultados...")
if os.path.exists(ZIP_FILENAME):
    os.remove(ZIP_FILENAME)

shutil.make_archive('MIR_Brutal_Analysis_Results', 'zip', OUTPUT_DIR)

print("\n--- ANALISIS COMPLETADO ---")
print(f"Archivo listo: {ZIP_FILENAME}")
display(FileLink(ZIP_FILENAME, result_html_prefix="Click para descargar: "))